In [1]:
%matplotlib inline

In [2]:
!pip install mxnet-cu101
!pip install gluoncv

In [3]:
import mxnet as mx
import numpy as np
import os, time, shutil
import matplotlib.pyplot as plt
from PIL import Image

from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs
from gluoncv.model_zoo import get_model

# 4. Transfer Learning with Your Own Image Dataset

Dataset size is a big factor in the performance of deep learning models.
``ImageNet`` has over one million labeled images, but
we often don't have so much labeled data in other domains.
Training a deep learning models on small datasets may lead to severe overfitting.

Transfer learning is a technique that addresses this problem.
The idea is simple: we can start training with a pre-trained model,
instead of starting from scratch.
As Isaac Newton said, "If I have seen further it is by standing on the
shoulders of Giants".


In [4]:
import os
from google.colab import files, drive   
import pandas as pd

# mount the google drive to my Colab session
drive.mount('/content/gdrive')
# use the google drive in my Colab session
print(os.listdir('/content/gdrive/My Drive/'))

home_path = '/content/gdrive/My Drive/cs101'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['cs101', 'Getting started.pdf', 'Colab Notebooks', 'embryo_13_fluo_sequence.mp4', 'embryo_13_fluo_slice.mp4', 'embryo_13_bf_slice.mp4', 'embryo_13_bf_sequence.mp4', 'hi', 'embryo data.gsheet', 'classify_fluo_pretrained.ipynb']


In [5]:
# Fixing the random seed
mx.random.seed(42)
np.random.seed(42)

# Available data
embryo_inds = [1, 3, 12, 13, 16, 18, 19, 24, 39, 40, 42, 45, 46, 47, 49, 50, 52, 53]

# Load info about videos
video_time_info = pd.read_excel(f'{home_path}/video_data_time_info_CS101.xlsx', index_col=0, header=0, na_values=['NaN','NAN'], usecols=['usable_index', 'first_pol_time', 't_num'])  
video_time_info.dropna(inplace=True, subset=['first_pol_time'])
print(video_time_info.loc[embryo_inds, 't_num'])

# Directory of the processed *.npy files
processed_path = f'{home_path}/processed'
polar_processed_path = f'{processed_path}/polarization'

p = np.random.permutation(len(embryo_inds))
p_embryo = [embryo_inds[i] for i in p]
t_num = list(video_time_info.loc[embryo_inds, 't_num'])
t_num_random = list(video_time_info.loc[p_embryo, 't_num'])

instance_cum_random = np.cumsum(t_num_random)
test_split_point = instance_cum_random[-1]*0.83
temp = abs(instance_cum_random-test_split_point)
test_idx = np.argmin(temp)

val_split_point = instance_cum_random[-1]*0.7
temp = abs(instance_cum_random-val_split_point)
val_idx = np.argmin(temp)

# normal t list grouped by embryo
# instance_cum = list(np.cumsum(t_num))
# instance_cum = [0] + instance_cum
# t_list = {};
# for k in range(len(instance_cum)-1):
#     t_list[k] = list(range(int(instance_cum[k]),int(instance_cum[k+1])))
# # randomized t list grouped by embryo 
# train_list_random = []
# for k in p[0:idx]:
#     train_list_random += t_list[k]

# test_list_random = []
# for k in p[idx+1:]:
#     test_list_random += t_list[k]

# print(train_list_random)
# print(test_list_random)
# Output 

train_embryos = p_embryo[:val_idx]
val_embryos = p_embryo[val_idx:test_idx]
test_embryos = p_embryo[test_idx:]
print(train_embryos)
print(val_embryos)
print(test_embryos)

data_path = f'{processed_path}/fluo_data/middle'
pol_path = f'{processed_path}/polarization'
train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'val')
test_path = os.path.join(data_path, 'test')

usable_index
1      21.0
3      21.0
12    143.0
13    143.0
16    143.0
18    143.0
19    143.0
24     21.0
39     21.0
40     21.0
42     21.0
45     21.0
46     21.0
47     21.0
49     21.0
50     21.0
52     21.0
53     21.0
Name: t_num, dtype: float64
[1, 3, 39, 18, 13, 47, 52, 50, 45, 12, 40, 53]
[16, 46, 24]
[42, 49, 19]


In [23]:
# Actually create the images
for i in range(len(train_embryos)):
  embryo_idx = train_embryos[i]
  embryo_path = f'{data_path}/embryo_{embryo_idx}.npy'
  embryo_pol_path = f'{pol_path}/embryo_{embryo_idx}.npy'
  embryo = np.load(embryo_path)
  embryo_pol = np.squeeze(np.load(embryo_pol_path)).astype(int)
  embryo = embryo.astype(np.float64) / np.max(embryo) # normalize the data to 0 - 1
  embryo = 255 * embryo # Now scale by 255
  embryo = embryo.astype(np.uint8)
  print(embryo_idx)
  for t in range(np.shape(embryo)[2]):
    pol = embryo_pol[t]
    img = Image.fromarray(embryo[:,:,t], 'L')
    img_path = f'{train_path}/{pol}/embryo_{embryo_idx}_{t}.png'
    img.save(img_path)

for i in range(len(val_embryos)):
  embryo_idx = val_embryos[i]
  embryo_path = f'{data_path}/embryo_{embryo_idx}.npy'
  embryo_pol_path = f'{pol_path}/embryo_{embryo_idx}.npy'
  embryo = np.load(embryo_path)
  embryo_pol = np.squeeze(np.load(embryo_pol_path)).astype(int)
  embryo = embryo.astype(np.float64) / np.max(embryo) # normalize the data to 0 - 1
  embryo = 255 * embryo # Now scale by 255
  embryo = embryo.astype(np.uint8)
  print(embryo_idx)
  for t in range(np.shape(embryo)[2]):
    pol = embryo_pol[t]
    img = Image.fromarray(embryo[:,:,t], 'L')
    img_path = f'{val_path}/{pol}/embryo_{embryo_idx}_{t}.png'
    img.save(img_path)

for i in range(len(test_embryos)):
  embryo_idx = test_embryos[i]
  embryo_path = f'{data_path}/embryo_{embryo_idx}.npy'
  embryo_pol_path = f'{pol_path}/embryo_{embryo_idx}.npy'
  embryo = np.load(embryo_path)
  embryo_pol = np.squeeze(np.load(embryo_pol_path)).astype(int)
  embryo = embryo.astype(np.float64) / np.max(embryo) # normalize the data to 0 - 1
  embryo = 255 * embryo # Now scale by 255
  embryo = embryo.astype(np.uint8)
  print(embryo_idx)
  for t in range(np.shape(embryo)[2]):
    pol = embryo_pol[t]
    img = Image.fromarray(embryo[:,:,t], 'L')
    img_path = f'{test_path}/{pol}/embryo_{embryo_idx}_{t}.png'
    img.save(img_path)

1
3
39
18
13
47
52
50
45
12
40
53
16
46
24
42
49
19


## Hyperparameters

First, let's import all other necessary libraries.



We set the hyperparameters as following:



In [12]:
classes = 2

epochs = 15
lr = 0.001
per_device_batch_size = 16
momentum = 0.9
wd = 0.0001

lr_factor = 0.75
lr_steps = [10, 20, 30, np.inf]

num_gpus = 1
num_workers = 8
ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
batch_size = per_device_batch_size * max(num_gpus, 1)
print(ctx)

[gpu(0)]


Things to keep in mind:

1. ``epochs = 5`` is just for this tutorial with the tiny dataset. please change it to a larger number in your experiments, for instance 40.
2. ``per_device_batch_size`` is also set to a small number. In your experiments you can try larger number like 64.
3. remember to tune ``num_gpus`` and ``num_workers`` according to your machine.
4. A pre-trained model is already in a pretty good status. So we can start with a small ``lr``.

## Data Augmentation

In transfer learning, data augmentation can also help.
We use the following augmentation in training:

2. Randomly crop the image and resize it to 224x224
3. Randomly flip the image horizontally
4. Randomly jitter color and add noise
5. Transpose the data from height*width*num_channels to num_channels*height*width, and map values from [0, 255] to [0, 1]
6. Normalize with the mean and standard deviation from the ImageNet dataset.




In [7]:
transform_train = transforms.Compose([
    transforms.Resize(600, keep_ratio=True),
    transforms.CenterCrop(512),

    transforms.RandomFlipLeftRight(), # Randomly flip the image horizontally
    transforms.RandomFlipTopBottom(),
    transforms.RandomLighting(0.1), # Add AlexNet-style PCA-based noise to an image
    transforms.RandomContrast(0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize(600, keep_ratio=True),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


With the data augmentation functions, we can define our data loaders:



In [8]:
data_path = f'{processed_path}/fluo_data/middle'
train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'val')
test_path = os.path.join(data_path, 'test')

train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(train_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(val_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers = num_workers)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(test_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

print(len(train_data) + len(val_data) + len(test_data))

63


Note that only ``train_data`` uses ``transform_train``, while
``val_data`` and ``test_data`` use ``transform_test`` to produce deterministic
results for evaluation.

## Model and Trainer

We use a pre-trained ``ResNet50_v2`` model, which has balanced accuracy and
computation cost.



In [9]:
model_name = 'ResNet50_v2'
finetune_net = get_model(model_name, pretrained=True)
with finetune_net.name_scope():
    finetune_net.output = nn.Dense(classes)
finetune_net.output.initialize(init.Xavier(), ctx = ctx)
finetune_net.collect_params().reset_ctx(ctx)
finetune_net.hybridize()

trainer = gluon.Trainer(finetune_net.collect_params(), 'sgd', {
                        'learning_rate': lr, 'momentum': momentum, 'wd': wd})
metric = mx.metric.Accuracy()
L = gluon.loss.SoftmaxCrossEntropyLoss()

Here's an illustration of the pre-trained model
and our newly defined model:

|image-model|

Specifically, we define the new model by::

1. load the pre-trained model
2. re-define the output layer for the new task
3. train the network

This is called "fine-tuning", i.e. we have a model trained on another task,
and we would like to tune it for the dataset we have in hand.

We define a evaluation function for validation and testing.



In [10]:
def test(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        metric.update(label, outputs)

    return metric.get()

## Training Loop

Following is the main training loop. It is the same as the loop in
`CIFAR10 <dive_deep_cifar10.html>`__
and ImageNet.

<div class="alert alert-info"><h4>Note</h4><p>Once again, in order to go through the tutorial faster, we are training on a small
    subset of the original ``MINC-2500`` dataset, and for only 5 epochs. By training on the
    full dataset with 40 epochs, it is expected to get accuracy around 80% on test data.</p></div>



In [ ]:
lr_counter = 0
num_batch = len(train_data)
train_acc_lst = []
train_loss_lst = []
val_acc_lst = []

for epoch in range(epochs):
    if epoch == lr_steps[lr_counter]:
        trainer.set_learning_rate(trainer.learning_rate*lr_factor)
        lr_counter += 1

    tic = time.time()
    train_loss = 0
    metric.reset()

    for i, batch in enumerate(train_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        with ag.record():
            outputs = [finetune_net(X) for X in data]
            loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
        for l in loss:
            l.backward()

        trainer.step(batch_size)
        train_loss += sum([l.mean().asscalar() for l in loss]) / len(loss)

        metric.update(label, outputs)

    _, train_acc = metric.get()
    train_loss /= num_batch

    _, val_acc = test(finetune_net, val_data, ctx)

    train_acc_lst.append(train_acc)
    train_loss_lst.append(train_loss)
    val_acc_lst.append(val_acc)
    print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc: %.3f | time: %.1f' %
             (epoch, train_acc, train_loss, val_acc, time.time() - tic))

_, test_acc = test(finetune_net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))

[Epoch 0] Train-acc: 0.992, loss: 0.028 | Val-acc: 0.941 | time: 37.2


## Next

Now that you have learned to muster the power of transfer
learning, to learn more about training a model on
ImageNet, please read `this tutorial <dive_deep_imagenet.html>`__.

The idea of transfer learning is the basis of
`object detection <../examples_detection/index.html>`_ and
`semantic segmentation <../examples_segmentation/index.html>`_,
the next two chapters of our tutorial.

.. |image-minc| image:: https://raw.githubusercontent.com/dmlc/web-data/master/gluoncv/datasets/MINC-2500.png
.. |image-model| image:: https://zh.gluon.ai/_images/fine-tuning.svg

